In [1]:
import numpy as np
import pandas as pd
import plotly.plotly as py
import plotly.graph_objs as go

In [2]:
# Create some arrays of possible distributionsm 
# in the form of ([[multiples], [associated probabilities])
levine1 = np.array([[0,2,4,7.5,10], [0.5,0.23,0.11,0.08,0.08]])
levine2 = np.array([[0,2,4,7.5,10], [0.56,0.24,0.07,0.06,0.07]])
correlation = np.array([[0,3,7.5,15,35,50], [0.648,0.253,0.059,0.025,0.011,0.004]])
mcclure = np.array([[0,0.5,3,15,50], [0.5,0.25,0.18,0.05,0.02]])
wilson = np.array([[0,1,3,10], [0.4,0.3,0.2,0.1]])
dummy = np.array([[0,5,10,20], [0.7,0.1,0.1,0.1]])

# function to add a cumulative distribution key
def cumulator(dist):
    holder = 0
    cumulative = []
    for i in dist[1]:
        holder += i
        cumulative.append(holder)
    dist = np.insert(dist,2,cumulative, axis=0)
    return dist

levine1 = cumulator(levine1)
levine2 = cumulator(levine2)
correlation = cumulator(correlation)
mcclure = cumulator(mcclure)
wilson = cumulator(wilson)
dummy = cumulator(dummy)

In [3]:
mcclure

array([[0.0e+00, 5.0e-01, 3.0e+00, 1.5e+01, 5.0e+01],
       [5.0e-01, 2.5e-01, 1.8e-01, 5.0e-02, 2.0e-02],
       [5.0e-01, 7.5e-01, 9.3e-01, 9.8e-01, 1.0e+00]])

In [4]:
# function to run a simulated portfolio
def run_portfolio(dist, iterations, number_of_investments):
    total_outcome = []
    # function takes the dist and an array of rand numbers to match to multiple
    def multiple_selector(dist, outcome):
        fund_proceeds = []
        for x in outcome:
            for i in range(0, len(dist[0])):
                if x <= dist[2][i]:
                    fund_proceeds.append(dist[0][i])
                    break
                else:
                    continue
        return fund_proceeds
    
    # iterate through the number of portfolios and append to an average gross multiple for each
    for _ in range(0, iterations):
        rands = np.random.rand(number_of_investments)
        fund_proceeds = multiple_selector(dist, rands)
#         average = sum(fund_proceeds)/len(fund_proceeds)
        total_outcome.append(fund_proceeds)

        
    return total_outcome

In [5]:
test = run_portfolio(mcclure, 1, 50)
test


[[0.0,
  0.0,
  0.0,
  0.0,
  3.0,
  0.5,
  0.5,
  0.0,
  0.5,
  0.0,
  0.5,
  0.0,
  0.5,
  0.0,
  0.0,
  0.0,
  0.0,
  0.5,
  0.5,
  0.0,
  50.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.5,
  0.5,
  0.0,
  0.0,
  3.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.5,
  0.5,
  0.5,
  0.5,
  3.0,
  3.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.5]]

In [6]:
# run various simulations accross the assumptions
levine1_concentrated = run_portfolio(levine1, 10000, 20)
levine1_spread = run_portfolio(levine1, 10000, 200)
levine2_concentrated = run_portfolio(levine2, 10000, 20)
levine2_spread = run_portfolio(levine2, 10000, 200)
correlation_concentrated = run_portfolio(correlation, 10000, 20)
correlation_spread = run_portfolio(correlation, 10000, 200)
mcclure_concentrated = run_portfolio(mcclure, 10000, 20)
mcclure_spread = run_portfolio(mcclure, 10000, 200)
wilson_concentrated = run_portfolio(wilson, 10000, 20)
wilson_spread = run_portfolio(wilson, 10000, 200)

concentrated_ports = [levine1_concentrated, levine2_concentrated, correlation_concentrated, mcclure_concentrated, wilson_concentrated]
spread_ports = [levine1_spread, levine2_spread, correlation_spread, mcclure_spread, wilson_spread]

In [7]:
# chart the concentrated portfolios with 20 investments
trace1 = go.Histogram(x=mcclure_concentrated, opacity=0.75, name='Mcclure - 20')
trace2 = go.Histogram(x=levine1_concentrated, opacity=0.75, name='Levine1 - 20')
trace3 = go.Histogram(x=correlation_concentrated, opacity=0.75, name='Correlation - 20')
trace4 = go.Histogram(x=levine2_concentrated, opacity=0.75, name='Levine2 - 20',)
trace5 = go.Histogram(x=wilson_concentrated, opacity=0.75, name='Wilson - 20')

data = [trace1, trace2, trace3, trace4, trace5]
layout = go.Layout(barmode='overlay')
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='concentrated')

/Users/taylorclauson/anaconda3/lib/python3.6/site-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



In [8]:
# chart the portfolios with 200 investments
trace5 = go.Histogram(x=mcclure_spread, opacity=0.75, name='Mcclure - 200')
trace6 = go.Histogram(x=levine1_spread, opacity=0.75, name='Levine1 - 200')
trace7 = go.Histogram(x=correlation_spread, opacity=0.75, name='Correlation - 200')
trace8 = go.Histogram(x=levine2_spread, opacity=0.75, name='Levine2 - 200')
trace9 = go.Histogram(x=wilson_spread, opacity=0.75, name='Wilson - 200')

data = [trace5, trace6, trace7, trace8, trace9]
layout = go.Layout(barmode='overlay')
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='spread')

In [24]:
# function to turn the results into a df
def organize(portfolios):
    df = pd.DataFrame(columns=['% above 1.0', '% above 2.0', '% above 3.0'])
    for portfolio in portfolios:
        above1 = len([1 for i in portfolio if i > 1.0])
        percentage1 = above1/len(portfolio)
        above2 = len([1 for i in portfolio if i > 2.0])
        percentage2 = above2/len(portfolio)
        above3 = len([1 for i in portfolio if i > 3.0])
        percentage3 = above3/len(portfolio)
        stats = pd.Series([percentage1, percentage2, percentage3], index=['% above 1.0', '% above 2.0', '% above 3.0'])
        df = df.append(stats, ignore_index=True)
    return df

In [25]:
test = organize(spread_ports)
test.T

,0,1,2,3,4
% above 1.0,1.0000,1.0000,0.9999,0.9994,1.0000
% above 2.0,0.9080,0.3243,0.6442,0.7753,0.3028
% above 3.0,0.0012,0.0000,0.0257,0.1392,0.0000


In [26]:
test1 = organize(concentrated_ports)
test1.T

,0,1,2,3,4
% above 1.0,0.9736,0.9142,0.8476,0.7949,0.9277
% above 2.0,0.6419,0.4198,0.4491,0.4805,0.4071
% above 3.0,0.1587,0.0601,0.2176,0.3268,0.0549


In [36]:
wilson1 = run_portfolio(correlation, 10000, 1)
wilson5 = run_portfolio(correlation, 10000, 20)
wilson10 = run_portfolio(correlation, 10000, 40)
wilson20 = run_portfolio(correlation,10000,60)
wilson50 = run_portfolio(correlation,10000,80)
wilson100 = run_portfolio(correlation, 10000, 100)


In [37]:
wilsons = [wilson1, wilson5, wilson10, wilson20, wilson50, wilson100]
summary = organize(wilsons)
summary.T

,0,1,2,3,4,5
% above 1.0,0.3503,0.8589,0.9537,0.9831,0.9921,0.9962
% above 2.0,0.3503,0.4570,0.5156,0.5405,0.5584,0.5806
% above 3.0,0.1029,0.2219,0.1610,0.1226,0.0938,0.0720


In [38]:
for size in wilsons:
    above1 = len([1 for i in size if i > 3.0])
    percentage = above1/len(size)
    print(percentage)

0.1029
0.2219
0.161
0.1226
0.0938
0.072
